Quá trình train: https://colab.research.google.com/drive/13DiVWglmy8eoq1DigU84-Kbs8U4pBB2S?usp=sharing

Dữ liệu train: https://drive.google.com/drive/folders/1Chyu5mqgz4rDOuKLoJfxs750T5gXDpdL?usp=sharing

In [6]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np

In [7]:
class_list = ['pig']
cap=cv2.VideoCapture('PenB.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (1020, 500))

Lấy tọa độ của vùng cần detect

In [3]:
def get_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Khi nhấp chuột trái
        print(f"Toạ độ điểm: ({x}, {y})")

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame=cv2.resize(frame,(1000,500))

    
    cv2.imshow('Image', frame)
    
    # Gắn hàm callback vào cửa sổ
    cv2.setMouseCallback('Image', get_coordinates)
    
    if cv2.waitKey(1) & 0xFF == 27:  # Nhấn phím ESC để thoát
        break

cap.release()
cv2.destroyAllWindows()

Detect và Tracking

In [8]:
model = YOLO("best.pt") #best.pt là mô hình được train sẵn trên google colab
tracker = Tracker()
count = 0
i=0
while True:
    ret,frame = cap.read() #read() trả về 2 giá trị là ret và frame. ret có thể True hoặc False, frame là khung hình hiện tại
    if not ret:
        break
    frame=cv2.resize(frame,(1020,500)) #Resize lại frame để hiển thị vừa với màn hình

    #Mục đích của đoạn này là giúp giới hạn lại phạm vi mà mô hình cần detect trong trường hợp này phạm vi là hình thang
    pts = np.array([[329, 2], [678, 3], [57, 469], [941, 476]], np.int32)
    pts = pts.reshape((-1, 1, 2))
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [pts], 255)
    roi = cv2.bitwise_and(frame, frame, mask=mask)

    results=model.predict(roi) # Detect heo trong frame hiện tại
    a=results[0].boxes.data # Lấy ra thông tin bounding box
    px=pd.DataFrame(a).astype("float")
    list=[] #Danh sách lưu trữ các tọa độ của bounding box

    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'pig' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        max = len(bbox_id) #Số lượng heo là số lượng bounding box mà mô hình detect được
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 1) #Vẽ bounding box cho đối tượng
        cv2.putText(frame, "Pig" + str(id), (x3, y3 - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 255), 1) #Đặt tên cho đối tượng
        cv2.putText(frame, "Count:" + str(max), (125, 54), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 255), 1) #Hiển thị số lượng heo trong video
    out.write(frame)
    cv2.imshow("RGB", frame) # Hiển thị frame
    if cv2.waitKey(1)&0xFF==27: # Kết thúc bằng phím Esc
        break
cap.release()
out.release()
cv2.destroyAllWindows()


0: 320x640 13 pigs, 142.1ms
Speed: 13.7ms preprocess, 142.1ms inference, 15.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 pigs, 103.4ms
Speed: 0.0ms preprocess, 103.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 14 pigs, 111.4ms
Speed: 0.0ms preprocess, 111.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 pigs, 107.0ms
Speed: 2.3ms preprocess, 107.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 pigs, 101.8ms
Speed: 2.0ms preprocess, 101.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 pigs, 93.8ms
Speed: 3.1ms preprocess, 93.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 pigs, 101.1ms
Speed: 2.9ms preprocess, 101.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 13 pigs, 93.0ms
Speed: 1.4ms preprocess, 93.0ms inference, 0.0ms postprocess per image at shape (1,